<a href="https://colab.research.google.com/github/btaitel/flaming-octo-nemesis/blob/master/2022-09-01a-scans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part A: Accumulate Exercise #

_(September 1, 2022)_

## Recall: `accumulate` from `itertools` ##

Recall that the [`itertools` module](https://docs.python.org/3/library/itertools.html) includes a function, `accumuate`, that computes cumulative sums:

In [1]:
from itertools import accumulate

def cumulative_sum(x):
    return list(accumulate(x))

x = [5, 3, -4, 20, 2, 9, 0, -1]
cumulative_sum(x)

[5, 8, 4, 24, 26, 35, 35, 34]

> The `accumulate` function returns a _generator_. So to get the actual elements of the sequence, you need to "run" the generator. The implementation above does that by calling the `list` constructor.

The `accumulate` function can be used in more general settings than sums. For example, it can be used to do cumulative multiplies via the `func` parameter. The value of a `func` argument should be a function that can combine two elements. For example, a _cumulative product_, where the values are multiplied, would look like the following:

In [2]:
print("* Sums:", *accumulate(x)) # cumulative sums

def multiply(a, b):
    return a * b

print("* Products:", *accumulate(x, func=multiply)) # cumulative products!

* Sums: 5 8 4 24 26 35 35 34
* Products: 5 15 -60 -1200 -2400 -21600 0 0


The `func` parameter, which is intended to refer to a _function_ (not a function's value when evaluated at a specific input), makes `accumulate` an example of a **higher-order function.** That is, it's a function that can accept another function or return a new function.

Why is that useful? It makes it possible to write one function that can be customized by the user through a simple interface. In this case, we can accumulate _anything_ as long we use a _binary operator_ to combine partial accumulated values with new values. By "binary operator," we really just mean some function `f(a,b)` that accepts two inputs and produces an output of the same type.

> The idea of higher-order functions and general operators can be viewed as _applied_ abstract algebra, which we would agree sounds a bit like an oxymoron.

## Exercise: When to buy a stock? ##

Suppose you have the price of a stock on $n$ consecutive days. For example, here is a list of stock prices observed on 14 consecutive days (assume these are numbered from 0 to 13, corresponding to the indices):

```python
prices = [13, 11, 10, 8, 5, 8, 9, 6, 7, 7, 10, 7, 4, 3]
```

Suppose you buy on day `i` and sell on day `j`, where `j > i`. Then `prices[j] - prices[i]` measures your _profit_ (or _loss_, if negative).

**Your task.** Implement a function, `max_profit(prices)`, to compute the best possible profit you could have made given a list of prices.

```python
prices = [13, 11, 10, 8, 5, 8, 9, 6, 7, 7, 10, 7, 4, 3]
```
In the example, that profit turns out to be **5**. That's because you can buy on day 4, whose price is `prices[4] == 5`, and then sell on day 10, whose price is `prices[10] == 10`, yielding a profit of 10-5=5. It turns out there is no other combination will beat that profit.

There are two constraints on your solution:
1. You must use `accumulate()`. There is a (relatively) simple and fast solution that does so.
2. If only a loss is possible, your function should return 0.

#### Strategy 0: Take it one day at a time ####

Here is one simple, but inefficient, approach, which is to look day-by-day.

At any given day, $j$, let $s_j$ be the stock's price on that day. Suppose you decide to _sell_ the stock that day. How much money could you have made? Let the lowest price on any day $i < j$ be $b$. You can find $b$ by looking for the minimum cost up to (but excluding) $s_j$. Then the best profit on day $j$, or its "gain," is $g_j = s_j - b$.

In [6]:
prices = [13, 11, 10, 8, 5, 8, 9, 6, 7, 7, 10, 7, 4, 3]

# Prototype something here #
day = 4 # j
sell = prices[day] # s — sell stock on this `day`
best_buy = min(prices[:day]) # b
gain = sell - best_buy
gain, max(gain, 0)

def max_gain_on_day(day, prices, verbose=False):
    sell = prices[day] # s — sell stock on this `day`
    if verbose: print("Day", day, "=> Sell at $", sell)
    best_buy = min(prices[:day]) # b
    if verbose: print("Preceding prices:", prices[:day], "=> best buy:", best_buy)
    gain = sell - best_buy
    if verbose: print("Gain:", gain, "=> Profit:", max(gain, 0))
    return max(gain, 0)
#this returns the max gain if you provide a specific day (zero base)
#and if you provide the stock prices
#and they worked some magic with a boolean where we can choose to use print
#statements or not

max_gain_on_day(5, prices, verbose=True)
#this is saying that i want to sell on day 5, and i want print statements

Day 5 => Sell at $ 8
Preceding prices: [13, 11, 10, 8, 5] => best buy: 5
Gain: 3 => Profit: 3


3

In [7]:
def max_profit__v0(prices):
    max_gain = 0
    for day in range(1, len(prices)): #if they choose day 0 the fx breaks, so we start at 1
        gain = max_gain_on_day(day, prices)
        if gain > max_gain:
            max_gain = gain
    return max_gain

# we are looping through a range because it gives us the ability to get the max for each day
# from our other function. then we are comapring that to max_gain of 0, in order
#to figure out which one is the max gain
#cause our first function you just pick a day but here we are looping through it

prices = [13, 11, 10, 8, 5, 8, 9, 6, 7, 7, 10, 7, 4, 3]
max_profit__v0(prices)

5

This solution works but has a flaw: it is cost (or work) inefficient! The call to `max_gain_on_day` could take as many as `n` steps to complete, and like our naïve cumulative sum, does redundant work as we go from day to day. You should be able to convince yourself that the cost of this method is, overall, $\mathcal{O}(n^2)$ for an input with just $n$ days.

#### Strategy 1: A cost-efficient method ####

The redudancy in Strategy 0 points toward a solution. At every day $j$, a useful piece of information is what is the lowest price observed _so far_. We can determine that for every day by calculating the _cumulative minimum price_ at every day. If we call that $s_j$, then the best gain on day $j$ is $g_j = c_j - s_j$, and then best overall gain is the largest of all $g_j$ values. In just a few passes over all the prices, we can, therefore, determine the maximum gain.

As an example, suppose the original prices are:
```python
prices =     [13, 11, 10, 8, 5, 8, 9, 6, 7, 7, 10, 7, 4, 3]
```

The minimum price on each day is:
```python
min_prices = [13, 11, 10, 8, 5, 5, 5, 5, 5, 5,  5, 5, 4, 3]
```
You can calculate those just by using **one** min-accumulate, which incurs a runtime cost of $\mathcal{O}(n)$ operations.

The best gain on _each day_ `i` is `prices[i] - min_prices[i]`, or:
```python
gains =      [ 0,  0,  0, 0, 0, 3, 4, 1, 2, 2,  5, 2, 0, 0]
```
Again, you can obtain in one pass over `prices` and `min_prices`. Looking at all these gains, the largest one is 5.

In [8]:
list(accumulate(prices, func=min))
#we dont have to create a min function, the itertools accumulate function knows what to do
#given that we pass in a list and func=min

#this is the cumulatiev min that we will use to see max profit

[13, 11, 10, 8, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3]

In [5]:
def max_profit__v1(prices):
    min_prices = list(accumulate(prices, func=min)) #cumulative min list
    gains = []
    for i in range(len(prices)): #using a range cuz technically they are looping through both lists at same time
        gains.append(prices[i] - min_prices[i])
    return max(gains)

prices = [13, 11, 10, 8, 5, 8, 9, 6, 7, 7, 10, 7, 4, 3]
max_profit__v1(prices)

5

#### Revisions for readability ####

The algorithmic solution of Strategy 1 is a very good one. To make it more readable, let's massage the loop to iterate over values in `prices` and `min_prices` directly.

To do so, observe that every time we access `prices[i]`, we access `min_prices[i]`, which occurs at the _same_ position in their respective lists. The "higher-level" concept here is that we are _pairing off_ elements of two collections of the same size. Therefore, we can iterate over them _in parallel_, which is an idiom called a **zipper iteration**. The name comes from the image of a zipper on your clothes, where the teeth are paired off.

In [10]:
#they are saying its even better just to zip them together and loop through that directly

def max_profit__v2(prices):
    min_prices = accumulate(prices, func=min)
    gains = [c - s for c, s in zip(prices, min_prices)]
    return max(gains)

prices = [13, 11, 10, 8, 5, 8, 9, 6, 7, 7, 10, 7, 4, 3]
max_profit__v2(prices)

5

In [11]:
# Test cell

max_profit = max_profit__v2

def check_profit(prices):
    print("\nTesting: prices={}".format(prices))
    profit_test = max_profit(prices)
    profit = max_profit(prices)
    print("\t==> The code's maximum profit: {}".format(profit))

    # Do an exhaustive search -- a correct, but highly inefficient, algorithm
    true_max = 0
    i_max, j_max = -1, -1
    for i in range(len(prices)):
        for j in range(i, len(prices)):
            gain_ij = prices[j] - prices[i]
            if gain_ij > true_max:
                i_max, j_max, true_max = i, j, gain_ij
    if i_max >= 0 and j_max >= 0:
        explain = "Buy on day {} at price {} and sell on {} at {}.".format(i_max, prices[i_max],
                                                                           j_max, prices[j_max])
    else:
        explain = "No buying options!"
    print("\t==> True max profit: {} ({})".format(true_max, explain))
    assert profit == true_max, "Your code's calculation does not match."

check_profit([13, 11, 10, 8, 5, 8, 9, 6, 7, 7, 10, 7, 4, 3])
check_profit([5, 4, 3, 2, 1])
check_profit([1, 2, 3, 4, 5])

for _ in range(8): # Random test cases
    from random import randint
    num_days = randint(1, 10)
    prices = [randint(1, 20) for _ in range(num_days)]
    check_profit(prices)

print("\n(Passed!)")


Testing: prices=[13, 11, 10, 8, 5, 8, 9, 6, 7, 7, 10, 7, 4, 3]
	==> The code's maximum profit: 5
	==> True max profit: 5 (Buy on day 4 at price 5 and sell on 10 at 10.)

Testing: prices=[5, 4, 3, 2, 1]
	==> The code's maximum profit: 0
	==> True max profit: 0 (No buying options!)

Testing: prices=[1, 2, 3, 4, 5]
	==> The code's maximum profit: 4
	==> True max profit: 4 (Buy on day 0 at price 1 and sell on 4 at 5.)

Testing: prices=[3, 20, 18, 7, 3, 14, 1]
	==> The code's maximum profit: 17
	==> True max profit: 17 (Buy on day 0 at price 3 and sell on 1 at 20.)

Testing: prices=[14, 14, 11, 11, 7, 15, 17, 7, 17]
	==> The code's maximum profit: 10
	==> True max profit: 10 (Buy on day 4 at price 7 and sell on 6 at 17.)

Testing: prices=[5, 15, 10, 16]
	==> The code's maximum profit: 11
	==> True max profit: 11 (Buy on day 0 at price 5 and sell on 3 at 16.)

Testing: prices=[8]
	==> The code's maximum profit: 0
	==> True max profit: 0 (No buying options!)

Testing: prices=[13, 18, 18]
	==

## Summary ##

Here are the key ideas to review from this notebook.

1. **Achieving cost (or work) efficiency.** When designing a computational algorithm that operates on $n$ input objects and produces $k$ outputs, a good goal is _linear_ scaling. That means the computational algorithm takes $\mathcal{O}(n+k)$ steps. In this case, doubling the input or output will double the computational cost, which seems like a reasonable price to pay.

2. **Improving readability through basic idioms.** Python has many constructs that have led to common coding conventions, or _idioms_. Learning these idioms helps improve the readability of your code by making it more concise and using patterns that should be familiar to other Python programmers. The examples you saw here included **slicing for lists**, **higher-level functions** (like `sum` and `accumulate`), **list comprehensions** (use judiciously!), **helper functions**, and **zipper iteration**.

3. **Higher-order functions.** A powerful idea in software development, which has its roots in abstract algebra, is that of higher-order functions. These are functions that take other functions as input and use them. List comprehensions are one example: the comprehension `[f(e) for e in x]` works for any function `f` that can take the input `e`. The use of `accumulate` customized to do additions (the default), multiplications, minimums, or whatever it is you need to calculate the Fibonacci sequence, are all additional examples.